In [122]:
import pandas as pd
import numpy as np
import json
import random

# Goal: Pick hero to maximize team's win % relative to what heroes have been picked and what heroes may be picked on each team

In [123]:
with open('test.json') as dataset, open('hero_list.json') as herolist:
    data = json.load(dataset)
    heroes = json.load(herolist)

In [124]:
withDB = pd.DataFrame(index = heroes, columns = heroes)
vsDB = pd.DataFrame(index = heroes, columns = heroes)

In [125]:
for hero in heroes:
    for other in heroes:
        withDB[hero][other] = data[hero]['winrate_with'][other]
        vsDB[hero][other] = data[hero]['winrate_vs'][other]

In [126]:
def pick_hero(heroesBanned, alliedHeroes, enemyHeroes, herolist=heroes):
    
    #unpickable heroes
    unpickable = heroesBanned + alliedHeroes + enemyHeroes
    # list of pickable heroes
    heroes = [hero for hero in herolist if hero not in unpickable]
    # ratio to determine how important considering unseen enemy heroes is
    unseen_factor = (1 - (len(enemyHeroes)/5))
    # array to store top 5 hero choices
    top_five = []
    
    # find top 5 heroes based on currently seen picks/bans
    for hero in heroes:
        
        # hero compatability with/vs seen picks
        compat = 0
        # sort top 5 heroes by compatibility
        top_five.sort(key=sort_key)
        
        # find current heroes compat
        for other in alliedHeroes:
            compat += withDB[hero][other]
        for other in enemyHeroes:
            compat += vsDB[hero][other]
        compat = compat/len(alliedHeroes + enemyHeroes)
        
        # add hero to top five if less than 5 heroes in list
        if(len(top_five) < 5):
            top_five.append([hero, compat])
        # replace least compatible hero with current hero if more compatible
        elif(top_five[0][1] < compat):
            top_five[0] = [hero, compat]
    
    # sort top 5 heroes based on remaining enemy picks possible
    for hero in top_five:
        
        # winrate of hero against potential enemy heroes
        unseen_wr = 0
        # number of enemy heroes counted
        count = 0

        # find winrate against unseen heroes
        for other in heroes:
            if(hero[0] != other):
                unseen_wr += vsDB[hero[0]][other]
                count += 1
        unseen_wr = unseen_wr/count
        
        # find overall winrate scaling value of unseen heroes by number of unseen enemy heroes
        hero[1] = (1-unseen_factor)*hero[1] + unseen_wr*unseen_factor
        
    # sort top 5 heroes
    top_five.sort(key=sort_key)
    
    # return top 5 hero suggestions
    return top_five

## Test1

In [127]:
allies = ['bane', 'queenofpain', 'centaur', 'shadow_shaman']
enemies = ['crystal_maiden', 'mirana', 'beastmaster', 'lina']
banned = []
pick_hero(banned, allies, enemies)

[['zuus', 0.5564874741174454],
 ['tinker', 0.5574540844298932],
 ['jakiro', 0.5590434108279668],
 ['drow_ranger', 0.5661084256109907],
 ['enigma', 0.5751904169556304]]

## Test2

In [128]:
allies = ['bane', 'queenofpain', 'centaur', 'shadow_shaman']
enemies = ['crystal_maiden', 'mirana', 'beastmaster', 'lina']
banned = ['zuus', 'enigma']
pick_hero(banned, allies, enemies)

[['bloodseeker', 0.5474202552614195],
 ['necrolyte', 0.5492343938370623],
 ['tinker', 0.5577649756774884],
 ['jakiro', 0.5592497055632166],
 ['drow_ranger', 0.5663689768991375]]

## Test3

In [129]:
allies = ['crystal_maiden', 'mirana', 'beastmaster', 'lina']
enemies = ['bane', 'queenofpain', 'centaur', 'shadow_shaman']
banned = []
pick_hero(banned, allies, enemies)

[['visage', 0.5413207338115444],
 ['silencer', 0.5454025744835254],
 ['huskar', 0.5471180941629028],
 ['abaddon', 0.5486170493459407],
 ['dazzle', 0.5489519176716043]]

## Test4

In [130]:
allies = ['crystal_maiden', 'mirana', 'beastmaster', 'lina']
enemies = ['bane', 'queenofpain', 'centaur', 'shadow_shaman']
banned = random.sample([hero for hero in heroes if hero not in (allies + enemies)], 10)
print('banned heroes: ' + str(banned))
pick_hero(banned, allies, enemies)

banned heroes: ['bounty_hunter', 'meepo', 'dark_willow', 'arc_warden', 'tinker', 'elder_titan', 'pugna', 'puck', 'treant', 'dazzle']


[['visage', 0.5418125819298584],
 ['drow_ranger', 0.5431126356698976],
 ['silencer', 0.5454322320031365],
 ['huskar', 0.5470393242679613],
 ['abaddon', 0.5489247855538026]]